In [1]:
"""
SIMULADOR DE EMPRÉSTIMO - (Loan Simulator)
Autor: José Wellington Albuquerque
https://www.linkedin.com/in/jwalbuquerque/

Data: 2025-02-17

Descrição:
Este código implementa um simulador de empréstimo com interface interativa com ipywidgets, gráficos dinâmicos via Plotly e tratamento de erros e monitoramento via Loguru.
Utiliza conceitos avançados como orientação a objetos, separação de preocupações e design pattern MVC implícito. 
Maiores informações em Design Patterns: 

Elements of Reusable Object-Oriented Software - Amazon: https://amzn.to/4k4j3Xm Ano 2021
Versão antiga em Português:
Padrões de Projetos: Soluções Reutilizáveis de Software Orientados a Objetos - Amazon: https://amzn.to/4i2kmnD
Aprendendo Padrões de Projeto em Python: Tire Proveito da Eficácia dos Padrões de Projeto (design Patterns) 
em Python Para Resolver Problemas do Mundo Real em Arquitetura e Design de Software - Amazon: https://amzn.to/4i3tgBg

Practical Python Design Patterns: Pythonic Solutions to Common Problems  - Amazon: https://amzn.to/42Z7cDO
Mastering Python Design Patterns - Third Edition: Craft essential Python patterns by following core design principles - Amazon: https://amzn.to/4gEJa44


"""
#Caso não queira usar a versão que verifica pacotes instalados, comente a linha abaixo
"""# Importações organizadas por tipo (bibliotecas padrão, terceiros, locais)
from typing import Tuple, List
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
"""

'# Importações organizadas por tipo (bibliotecas padrão, terceiros, locais)\nfrom typing import Tuple, List\nimport numpy as np\nimport plotly.graph_objects as go\nimport ipywidgets as widgets\nfrom IPython.display import display, clear_output\n'

In [2]:
# =================================================================================
#   INSTALA OS PACOTES NECESSÁRIOS PARA O FUNCIONAMENTO DO SCRIPT
# =================================================================================

"""
VERIFICA E INSTALA OS PACOTES: ipywidgets, numpy, plotly e IPython.
IMPORTAÇÕES ESPECÍFICAS INCLUEM TYPING, GRAPH_OBJECTS E DISPLAY.
"""

import importlib.util
import os
import sys
from loguru import logger
logger.debug("Debug das informações detalhadas para desenvolvimento")
logger.add("log_{time}.log")

def install(package):
    os.system(f"{sys.executable} -m pip install {package}")

# Define pacotes principais e aliases
required_packages = {
    'ipywidgets': {'alias': 'widgets', 'pkg_name': 'ipywidgets'},
    'numpy': {'alias': 'np', 'pkg_name': 'numpy'},
    'plotly': {'alias': None, 'pkg_name': 'plotly'},
    'IPython': {'alias': None, 'pkg_name': 'IPython'}
}

# Verifica e instala pacotes ausentes
for pkg_info in required_packages.values():
    pkg_name = pkg_info['pkg_name']
    if not importlib.util.find_spec(pkg_name):
        print(f"Instalando {pkg_name}...")
        install(pkg_name)
        if not importlib.util.find_spec(pkg_name):
            print(f"Falha ao instalar {pkg_name}. Encerrando.")
            sys.exit(1)

# Importações principais com aliases
for pkg_info in required_packages.values():
    pkg_name = pkg_info['pkg_name']
    alias = pkg_info['alias']
    try:
        if alias:
            exec(f"import {pkg_name} as {alias}")  # Ex: import numpy as np
        else:
            exec(f"import {pkg_name}")             # Ex: import plotly
    except Exception as e:
        print(f"Erro importando {pkg_name}: {e}")
        sys.exit(1)

# Importações específicas adicionais
try:
    from typing import Tuple, List
    import plotly.graph_objects as go
    from IPython.display import display, clear_output
except ImportError as e:
    print(f"Erro em importações específicas: {e}")
    sys.exit(1)

# Habilita extensões do Jupyter
os.system("jupyter nbextension enable --py widgetsnbextension")
os.system("jupyter nbextension enable --py plotlywidget")

print("Configuração concluída com sucesso!")


2025-02-17 18:28:07.714 | DEBUG    | __main__:<module>:14 - Debug das informações detalhadas para desenvolvimento


Configuração concluída com sucesso!


In [ ]:
# =================================================================================
# CONSTANTES GLOBAIS (Configuração centralizada)
# =================================================================================

def carregar_configuracoes():
    """Carrega e valida as configurações do empréstimo."""
    global DEFAULT_LOAN_AMOUNT, MIN_LOAN_AMOUNT, MAX_LOAN_AMOUNT, MAX_TAX_RATE, MAX_IOF_RATE

    while True:
        try:
            DEFAULT_LOAN_AMOUNT = round(float(input("Digite o valor do empréstimo: ")), 2)
            MIN_LOAN_AMOUNT = round(float(input("Digite o valor mínimo permitido: ")), 2)
            MAX_LOAN_AMOUNT = round(float(input("Digite o valor máximo permitido: ")), 2)
            MAX_TAX_RATE = round(float(input("Digite a taxa máxima de juros (%): ")), 2)
            MAX_IOF_RATE = round(float(input("Digite a taxa máxima do IOF (%): ")), 2)

            if not (MIN_LOAN_AMOUNT <= DEFAULT_LOAN_AMOUNT <= MAX_LOAN_AMOUNT):
                print("Erro: O valor do empréstimo deve estar entre o mínimo e o máximo.")
                continue

            if MAX_TAX_RATE < 0 or MAX_IOF_RATE < 0:
                print("Erro: Taxas não podem ser negativas.")
                continue

            break

        except ValueError:
            print("Erro: Insira um valor numérico válido.")

# Chamada para carregar as configurações do empréstimo
carregar_configuracoes()

# Configuração padrão (serão sobrescritas pela entrada do usuário)
# Esses valores iniciais permanecem apenas para referência caso não se utilize a entrada manual
# DEFAULT_LOAN_AMOUNT = 1000.0  # Valor padrão do empréstimo
# MIN_LOAN_AMOUNT = 100.0       # Valor mínimo permitido
# MAX_LOAN_AMOUNT = 10000.0     # Valor máximo permitido
# MAX_TAX_RATE = 9.95           # Taxa máxima de juros (%)
# MAX_IOF_RATE = 2.29           # Taxa máxima do IOF (%)

# Configurações de visualização do gráfico (DRY principle)
PLOT_CONFIG = {
    'colors': {
        'balance': '#1f77b4',   # Azul para saldo devedor
        'paid': '#2ca02c',      # Verde para total pago
        'charges': '#d62728',   # Vermelho para encargos
        'total': '#7f7f7f'      # Cinza para linha do total
    },
    'line_styles': {
        'total': 'dash'         # Linha tracejada para o total
    },
    'yaxis_margin': 50          # Margem superior do gráfico
}

# =================================================================================
# CLASSE PRINCIPAL (Encapsulamento da lógica)
# =================================================================================
class LoanSimulator:
    """
    Classe principal que gerencia toda a simulação do empréstimo.
    
    Responsabilidades:
    - Gerenciar widgets da interface
    - Executar cálculos financeiros
    - Gerar visualizações
    - Tratar eventuaus erros
    """
    
    def __init__(self) -> None:
        """Inicializa o simulador criando a interface e configurações iniciais."""
        self._output = widgets.Output()  # Área de output fixa para o gráfico
        self._initialize_widgets()       # Cria os controles deslizantes
        self._setup_ui()                 # Monta a interface do usuário

    # =============================================================================
    # MÉTODOS DE CONFIGURAÇÃO DA INTERFACE
    # =============================================================================
    def _initialize_widgets(self) -> None:
        """Configura os widgets interativos com validações e estilos."""
        # Estilo comum para padronização dos controles
        common_style = {'description_width': '120px'}
        layout = widgets.Layout(width='450px')

        # Widget para valor do empréstimo
        self.amount_slider = widgets.FloatSlider(
            value=DEFAULT_LOAN_AMOUNT,
            min=MIN_LOAN_AMOUNT,
            max=MAX_LOAN_AMOUNT,
            step=50.0,
            description='Valor Empréstimo:',
            style=common_style,
            layout=layout,
            readout_format='.2f'  # Formato monetário com duas casas
        )

        # Widget para número de parcelas (evita valores não inteiros)
        self.installments_slider = widgets.IntSlider(
            value=12,
            min=1,                # Não permite zero parcelas
            max=48,
            step=1,
            description='Nº de Parcelas:',
            style=common_style,
            layout=layout
        )

        # Widget para taxa de juros com limite máximo configurável
        self.tax_slider = widgets.FloatSlider(
            value=5.0,
            min=0.1,              # Evita taxa zero que quebraria cálculos
            max=MAX_TAX_RATE,
            step=0.1,
            description='Juros Mensal (%):',
            style=common_style,
            layout=layout,
            readout_format='.2f'  # Duas casas decimais para porcentagem
        )

        # Widget para IOF seguindo legislação brasileira
        self.iof_slider = widgets.FloatSlider(
            value=1.0,
            min=0.0,
            max=MAX_IOF_RATE,
            step=0.1,
            description='IOF (%):',
            style=common_style,
            layout=layout,
            readout_format='.2f'
        )

    def _setup_ui(self) -> None:
        """Organiza os componentes na tela e configura callbacks."""
        # Container vertical para os controles
        self.controls = widgets.VBox([
            self.amount_slider,
            self.installments_slider,
            self.tax_slider,
            self.iof_slider
        ])

        # Conecta a mudança nos widgets à atualização do gráfico
        widgets.interactive_output(self._update_display, self._widgets_dict)
        
        # Exibe a interface completa
        display(widgets.VBox([self.controls, self._output]))

    @property
    def _widgets_dict(self) -> dict:
        """Mapeamento dos widgets para o interactive_output (Design Pattern: Facade)."""
        return {
            'loan_amount': self.amount_slider,
            'installments': self.installments_slider,
            'tax_rate': self.tax_slider,
            'iof_rate': self.iof_slider
        }

    # =============================================================================
    # LÓGICA DE NEGÓCIO (Cálculos financeiros)
    # =============================================================================
    def _calculate_loan(self, loan_amount: float, installments: int, 
                       tax_rate: float, iof_rate: float) -> Tuple[float, float, float]:
        """
        Calcula os valores principais do empréstimo usando a fórmula da Tabela Price.
        
        Args:
            loan_amount: Valor solicitado pelo cliente
            installments: Número de parcelas
            tax_rate: Taxa de juros mensal (%)
            iof_rate: Taxa do IOF (%)
            
        Returns:
            Tupla com (valor contratado, valor total, valor da parcela)
            
        Raises:
            ValueError: Se qualquer valor for inválido
        """
        # Validação de inputs críticos, uso do zero ou valores negativos
        if any(v <= 0 for v in [loan_amount, installments, tax_rate]):
            raise ValueError("Todos os valores devem ser positivos")

        try:
            # Cálculo do IOF quando você pega o empréstimo, é aplicado o custo de IOF sobre valor que lhe emprestado
            iof_value = loan_amount * (iof_rate / 100)
            contracted_value = loan_amount + iof_value
            
            # Conversão da taxa para decimal
            monthly_tax = tax_rate / 100  
            
            # Fator de anuidade (Tabela Price)
            annuity_factor = (monthly_tax * (1 + monthly_tax)**installments) / (
                (1 + monthly_tax)**installments - 1)
            
            # Cálculo da parcela e valor total
            installment_value = contracted_value * annuity_factor
            total_value = installment_value * installments
            
            return contracted_value, total_value, installment_value
            
        except ZeroDivisionError:
            raise ValueError("Número de parcelas inválido") from None

    def _generate_amortization_table(self, contracted_value: float, installments: int, 
                                    tax_rate: float, installment_value: float) -> Tuple[List[float], List[float], List[float]]:
        """
        Gera a tabela de amortização mês a mês.
        
        Returns:
            Três listas com: saldo devedor, total pago, encargos acumulados
        """
        # Inicialização das variáveis de estado
        balance = [contracted_value]  # Saldo devedor inicial
        paid = [0.0]                  # Total pago inicial
        charges = [0.0]               # Encargos acumulados
        
        # Simulação mês a mês
        for _ in range(installments):
            current_balance = balance[-1]
            
            # Cálculo dos componentes da parcela
            interest = current_balance * (tax_rate / 100)  # Juros do período
            principal = installment_value - interest       # Amortização
            
            # Atualização dos valores
            new_balance = current_balance - principal
            balance.append(max(new_balance, 0))            # Evita saldo negativo
            paid.append(paid[-1] + installment_value)      # Acumula pagamento
            charges.append(charges[-1] + interest)         # Acumula encargos
            
        return balance, paid, charges

    # =============================================================================
    # VISUALIZAÇÃO (Lógica utilizada)
    # =============================================================================
    def _create_plot(self, balance: List[float], paid: List[float], 
                    charges: List[float], total_value: float) -> go.FigureWidget:
        """Cria o gráfico interativo usando Plotly."""
        months = np.arange(len(balance))  # Eixo X baseado no número de meses
        
        # Cria figura com atualização eficiente (FigureWidget)
        fig = go.FigureWidget()
        
        # Adiciona cada trace com configurações específicas
        fig.add_trace(go.Scatter(
            x=months, 
            y=balance,
            name='Saldo Devedor',
            line=dict(color=PLOT_CONFIG['colors']['balance']),
            hovertemplate='Mês %{x}<br>R$ %{y:.2f}<extra></extra>'  # Tooltip
        ))
        
        # Trace para total pago
        fig.add_trace(go.Scatter(
            x=months, 
            y=paid,
            name='Total Pago',
            line=dict(color=PLOT_CONFIG['colors']['paid']),
            hovertemplate='Mês %{x}<br>R$ %{y:.2f}<extra></extra>'
        ))
        
        # Trace para encargos
        fig.add_trace(go.Scatter(
            x=months, 
            y=charges,
            name='Encargos (Juros + IOF)',
            line=dict(color=PLOT_CONFIG['colors']['charges']),
            hovertemplate='Mês %{x}<br>R$ %{y:.2f}<extra></extra>'
        ))
        
        # Linha horizontal do valor total
        fig.add_trace(go.Scatter(
            x=months, 
            y=[total_value] * len(months),
            name='Valor Total Contrato',
            line=dict(
                color=PLOT_CONFIG['colors']['total'],
                dash=PLOT_CONFIG['line_styles']['total']
            ),
            hovertemplate='R$ %{y:.2f}<extra></extra>'
        ))
        
        # Configuração do layout do gráfico
        fig.update_layout(
            title='Simulador de Empréstimo: Projeção Detalhada de Custos e Amortização',
            xaxis_title='Mês',
            yaxis_title='Valor (R$)',
            hovermode='x unified',  # Mostra todos os valores no mesmo x
            yaxis_range=[0, total_value + PLOT_CONFIG['yaxis_margin']],  # Margem dinâmica
            legend=dict(orientation='h', yanchor='bottom', y=1.02),  # Legenda horizontal
            template='plotly_white'  # Tema limpo
        )
        
        return fig

    # =============================================================================
    # CONTROLE PRINCIPAL (Orquestração)
    # =============================================================================
    def _update_display(self, **kwargs) -> None:
        """Atualiza toda a exibição quando os parâmetros mudam."""
        with self._output:
            clear_output(wait=True)  # Limpa sem piscar
            
            try:
                # 1. Executa cálculos principais
                contracted, total, installment = self._calculate_loan(**kwargs)
                
                # 2. Gera tabela de amortização
                balance, paid, charges = self._generate_amortization_table(
                    contracted, kwargs['installments'], kwargs['tax_rate'], installment
                )
                
                # 3. Cria visualização
                fig = self._create_plot(balance, paid, charges, total)
                display(fig)  # Exibe o gráfico atualizado
                
                # 4. Exibe resumo numérico formatado
                print(f"► Valor Contratado: R$ {contracted:,.2f}".replace(',', '_').replace('.', ',').replace('_', '.'))
                print(f"► Valor Total: R$ {total:,.2f} ({kwargs['installments']}x)".replace(',', '_').replace('.', ',').replace('_', '.'))
                print(f"► Parcela Fixa: R$ {installment:,.2f}".replace(',', '_').replace('.', ',').replace('_', '.'))
                print(f"◼ Custo Efetivo Total (CET): {((total / kwargs['loan_amount'] - 1) * 100):.2f}%")
                
            except Exception as e:
                # Tratamento genérico de erros com feedback amigável
                print(f"⚠ Erro na simulação: {str(e)}")

    # =============================================================================
    # TESTES (Design for testability)
    # =============================================================================
    @staticmethod
    def test() -> None:
        """Executa testes unitários embutidos para validação."""
        test_cases = [
            (1000, 12, 5.0, 1.0),  # Caso padrão
            (2000, 24, 7.5, 2.0),  # Valores maiores
            (500, 0, 5.0, 1.0),     # Teste de erro esperado
        ]
        
        print("Iniciando testes...\n" + "-"*40)
        for case in test_cases:
            simulator = LoanSimulator()
            try:
                result = simulator._calculate_loan(*case)
                status = "OK" if result[0] > 0 else "FALHOU"
                print(f"Teste {case}: {status} → {result}")
            except Exception as e:
                print(f"Teste {case}: ERRO ESPERADO → {e}")
        print("-"*40 + "\nTestes concluídos!")

# =================================================================================
# EXECUÇÃO PRINCIPAL (Com tratamento global de exceções)
# =================================================================================
if __name__ == "__main__":
    try:
        logger.info("Iniciando aplicação principal")
        simulator = LoanSimulator()
        logger.success("Simulador carregado com sucesso")
    except Exception as e:
        logger.critical(f"Falha na inicialização: {e}")
        sys.exit(1)


2025-02-17 18:31:08.266 | INFO     | __main__:<module>:353 - Iniciando aplicação principal


2025-02-17 18:31:17.436 | SUCCESS  | __main__:<module>:355 - Simulador carregado com sucesso
